In [22]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

import datetime
import os
import shutil

from glob import glob
from copy import deepcopy

In [18]:
gpds = np.arange(10,121)
cols = ["bgk", "reg", "kbc"]
bcs = ["fwbb", "hwbb", "ibb1"]

base_path = "/home/mbille/Documents/cluster_hbrs_home/02_lbm_simulations/data_2D_GPD_BC_CO"

paths_dict = dict() # path matches
gpds_dict = dict() # lists of available gpds

# create path_dict: keys= bc_col Combination, values: lists of paths to gpd-variants
for col in cols:
    for bc in bcs:
        paths_dict[bc+"_"+col] = []
        gpds_dict[bc+"_"+col] = []
        for gpd in gpds:
            dir_names = glob(base_path+"/*2d_GPD"+str(gpd)+"_"+bc+"_"+col)
            if len(dir_names)>1:
                print("(!) Warning: more than 1 simulation for:", "2d_GPD"+str(gpd)+"_"+bc+"_"+col)
                print(dir_names)
            elif len(dir_names) == 1:
                # this is what we want: one unique simulation found
                paths_dict[bc+"_"+col].append(*dir_names)
                gpds_dict[bc+"_"+col].append(gpd)

            #print(base_path+"/*2d_GPD"+str(gpd)+"_"+bc+"_"+col)
#            print("2d_GPD"+str(gpd)+"_"+bc+"_"+col+":",dir_names)
#        print(paths_dict[bc+"_"+col])

#print(gpds)

In [37]:
# FIT SINE WAVE -> STROUHAL NUMBER
cols = ["bgk", "reg", "kbc"]
bcs = ["fwbb", "hwbb", "ibb1"]

filename = "/lift_coefficient.txt"
periodic_start=0.9

# function to fit
def sine_func(xx,a,b,c,d):
    return a*np.sin(2*np.pi*b*xx+c)+d

for col in cols:
    for bc in bcs:
        lift_St_fit = np.zeros((2,len(gpds_dict[bc+"_"+col])))
        lift_St_fit[0] = gpds_dict[bc+"_"+col]
        for gpd_i in range(len(lift_St_fit[0])):
            # load timeseries
            lift_timeseries = np.loadtxt(paths_dict[bc+"_"+col][gpd_i]+filename)
            lift_converged = lift_timeseries[int(lift_timeseries.shape[0]*periodic_start-1):]

            # fit sine_wave
            coefficients, sheeesh = curve_fit(sine_func, lift_converged[:,1], lift_converged[:,2], p0=(0.7,0.2,0.5,0))

            # plot fit and save to Desktop/St_Fits/...
            fig, ax = plt.subplots(constrained_layout=True)
            plt.plot(lift_converged[:,1], lift_converged[:,2], lift_converged[:,1], sine_func(lift_converged[:,1], *coefficients))
            plt.legend(["lift", "fit"])
            ax.set_xlabel("physical time / s")
            ax.set_ylabel("Coefficient of Lift")
            ax.set_ylim([-1,1])
            plt.savefig("/home/mbille/Desktop/St_Fits/St_LiftFit_2D_"+bc+"_"+col+"_GPD"+str(gpds_dict[bc+"_"+col][gpd_i])+".png")
            plt.close(fig)

            # store St
            lift_St_fit[1,gpd_i]=coefficients[1]
        np.savetxt("/home/mbille/Desktop/St_Fits/St_LiftFit_2D_"+bc+"_"+col+"-St(GPD).txt", lift_St_fit, header="2D, DpY19, D2Q9, "+bc+"_"+col)

In [40]:
# AVERAGE PEAKS OF LIFT -> Cl_max(mean)

#bcs= ["fwbb"]
#cols=["kbc"]

filename = "/lift_coefficient.txt"
periodic_start=0.75

for col in cols:
    for bc in bcs:
        lift_Cl_maxmean = np.zeros((2,len(gpds_dict[bc+"_"+col])))
        lift_Cl_maxmean[0] = gpds_dict[bc+"_"+col]
        for gpd_i in range(len(lift_Cl_maxmean[0])):
            # load timeseries
            lift_timeseries = np.loadtxt(paths_dict[bc+"_"+col][gpd_i]+filename)
            lift_converged = lift_timeseries[int(lift_timeseries.shape[0]*periodic_start-1):]

            # find peaks
            #peaks_max = find_peaks(lift_converged[:,2], prominence=lift_converged[:,2].max())  # prominence: "the minimum height necessary to descend to get from the summit to any higher terrain"
            peaks_max = find_peaks(lift_converged[:,2], prominence=lift_converged[:,2].max()-(lift_converged[:,2].max()+lift_converged[:,2].min())*0.5)
            lift_Cl_maxmean[1,gpd_i] = lift_converged[peaks_max[0],2].mean()

            # plot fit and save to Desktop/St_Fits/...
            fig, ax = plt.subplots(constrained_layout=True)
            ax.plot(lift_timeseries[:,1], lift_timeseries[:,2])
            plt.scatter(lift_converged[peaks_max[0],1],lift_converged[peaks_max[0],2],marker="+", facecolors="r")
            plt.axhline(y=lift_Cl_maxmean[1,gpd_i], color="r", ls="--", lw=0.5)
            plt.legend(["lift", "Cl_maxmean"])
            ax.set_xlabel("physical time / s")
            ax.set_ylabel("Coefficient of Lift")
            ax.set_ylim([-1,1])
            plt.savefig("/home/mbille/Desktop/Cl_Peaks/Cl_Peaks_2D_"+bc+"_"+col+"_GPD"+str(gpds_dict[bc+"_"+col][gpd_i])+".png")
            plt.close(fig)
        np.savetxt("/home/mbille/Desktop/Cl_Peaks/Cl_Peaks_2D_"+bc+"_"+col+"-Cl(GPD).txt", lift_Cl_maxmean, header="2D, DpY19, D2Q9, "+bc+"_"+col)

In [35]:
lift_converged[:,2].min()

-0.7215750477431399

In [34]:
lift_converged[:,2].max()

0.7213720431780384